## 5.3 Iterative Methods

In [ ]:
import numpy as np

We implement the basic power iteration as follows

In [ ]:
def power_iteration(A, x, n, k=0):
    assert (A.shape[0]==A.shape[1]), "Matrix not quadratic"
    assert (A.shape[0]==x.shape[0]), "Matrix and vector dimensions don't match"
    
    x0, x1 = x.copy(), np.zeros_like(x)
    lams = np.zeros(n)
    for i in range(n):
        x1[:] = np.inner(A, x0)
        lams[i] = x1[k] / x0[k]
        x0[:] = x1 / np.linalg.norm(x1)
    
    return lams

#### Example 5.13 (Power Iteration)

We consider the matrix
$$A = \begin{pmatrix} 2&1&2\\ -1 & 2 & 1 \\ 1 & 2 & 4 \end{pmatrix}$$
and the initial vector $x_0 = (1,1,1)^T$. Six iterations of the power method yield

In [ ]:
A = np.array([[2, 1, 2],
              [-1, 2, 1],
              [1, 2, 4]], dtype=np.double)
x = np.array([1, 1, 1], dtype=np.double)

l = power_iteration(A, x, 6, k=0)
print(f'l_{len(l)} = {l[-1]}')

Using `numpy.linalg`s functionality to compute Eigenvalues, we get an error of

In [ ]:
print(f'err = {abs(l[-1] - np.linalg.eig(A)[0][2]):.4e}')

To determine other Eigenvalues, we implement the inverse iteration with shift. Here, we re-use our implementation of the LU factorization with pivoting.

In [ ]:
from scripts.lu import lu_pivot, forward, backward

In [ ]:
def inverse_interation(A, x, n, sigma=0, k=0):
    n1 = A.shape[0]
    assert (n1==A.shape[1]), "Matrix not quadratic"
    assert (n1==x.shape[0]), "Matrix and vector dimensions don't match"
    
    B = np.array(A - sigma * np.identity(n1), dtype=A.dtype)
    pivot = lu_pivot(B)
    
    x0, x1, y = x.copy(), np.zeros_like(x), np.zeros_like(x)
    lams = np.zeros(n)

    for i in range(n):
        xk = x0[k]
        
        for p in pivot:
            x0[p] = x0[[p[1], p[0]]]
        y[:] = forward(B, x0)
        x1[:] = backward(B, y)

        mu = x1[k] / xk
        lams[i] = sigma + 1 / mu
        x0[:] = np.array(x1 / np.linalg.norm(x1, ord=np.inf))
    
    return lams

#### Example 5.15 (Inverse iteration with Shift)

We consider the same matrix. After four iterations of the inverse iteration with shifts $\sigma=2, -1, 4$, we get

In [ ]:
A = np.array([[2, -0.1, 0.4], [0.3, -1, 0.4], [0.2, -0.1, 4]])
v = np.array([1.0, 1.0, 1.0])

lam1 = inverse_interation(A, v, 4, 2, k=0)
lam2 = inverse_interation(A, v, 4, -1, k=0)
lam3 = inverse_interation(A, v, 4, 4, k=0)

print(f'lam_1^({len(lam1)}) = {lam1[-1]}')
print(f'lam_2^({len(lam2)}) = {lam2[-1]}')
print(f'lam_3^({len(lam3)}) = {lam3[-1]}')

Comparing with the approximation computed with `numpy`, we have

In [ ]:
lam = np.linalg.eig(A)[0]
print(f'err(lam_1) = {abs(lam1[-1] - lam[1]):.3e}')
print(f'err(lam_2) = {abs(lam2[-1] - lam[0]):.3e}')
print(f'err(lam_3) = {abs(lam3[-1] - lam[2]):.3e}')